In [1]:
import sys
sys.path

['D:\\Anaconda\\envs\\position_completion_new\\python36.zip',
 'D:\\Anaconda\\envs\\position_completion_new\\DLLs',
 'D:\\Anaconda\\envs\\position_completion_new\\lib',
 'D:\\Anaconda\\envs\\position_completion_new',
 '',
 'D:\\Anaconda\\envs\\position_completion_new\\Lib\\site-packages',
 'D:\\Anaconda\\envs\\position_completion_new\\Lib\\site-packages\\win32',
 'D:\\Anaconda\\envs\\position_completion_new\\Lib\\site-packages\\win32\\lib',
 'D:\\Anaconda\\envs\\position_completion_new\\Lib\\site-packages\\Pythonwin',
 'D:\\Anaconda\\envs\\position_completion_new\\Lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\93973\\.ipython']

In [2]:
print(sys.version)
print(sys.executable)

3.6.13 |Anaconda, Inc.| (default, Mar 16 2021, 11:37:27) [MSC v.1916 64 bit (AMD64)]
D:\Anaconda\envs\position_completion_new\python.exe


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy.stats import spearmanr, pearsonr
from decimal import Decimal

In [4]:
import os
os.chdir("../")

In [5]:
matplotlib.rcParams['font.sans-serif']=['Times New Roman'] 

#### 1.读取数据

In [6]:
AShareEODPricesPivot = pd.read_csv('./Data/AShareEODPricesPivot.csv',index_col = 0) # 读取完整的股价数据
left_skt_price = pd.read_excel("./Data/LeftStkPricePivot.xlsx",index_col = 0) # 补充剩余的股价
ChinaMutualFundNAV = pd.read_csv("./Data/ChinaMutualFundNAV.csv") # 真实的基金净值数据
month_begin_end = pd.read_excel("./Data/MonthBeginEnd.xlsx",index_col = "MONTH_END_ALL") # 月初月末表

In [7]:
# 去除LOF基金
ChinaMutualFundNAV = ChinaMutualFundNAV.loc[~ChinaMutualFundNAV.loc[:,'F_INFO_WINDCODE'].map(lambda x: x[-2:] in ["SH","SZ"])]
ChinaMutualFundNAV.reset_index(inplace = True,drop = True)

In [8]:
all_stk_price = pd.concat([AShareEODPricesPivot,left_skt_price],axis = 1) # 所有涉及的股票的收盘价

In [9]:
stock_returns = all_stk_price.ffill().pct_change() # 去除空置数据

In [10]:
stock_returns.index = pd.to_datetime(stock_returns.index)

#### 2.计算净值偏移 

In [11]:
pos_dir = r"./Output/PositionCompletionDetail"

In [12]:
for root_dir, sub_dir, files in os.walk(pos_dir): # 第一个为起始路径，第二个为起始路径下的文件夹，第三个是起始路径下的文件。
#     print('root_dir:', root_dir)
#     print('sub_dir:', sub_dir)
#     print('files:', files)
    for file in files:
        if file.endswith('.xlsx'):
#         if file == '160613.xlsx': # 测试代码
            file_name = os.path.join(root_dir, file) # 构造绝对路径
            tmp_complete_position = pd.read_excel(file_name, index_col = 0)
            tmp_fund = file.replace(".xlsx",".OF") # 获取当前基金的代码
            
            print(f"processing {tmp_fund}")
            
            tmp_complete_position.fillna(0,inplace = True) # 将空权重设置为0
            tmp_complete_position.index = month_begin_end.loc[tmp_complete_position.index,:].MONTH_END_TRADE.tolist() # 解决交易日问题
            
            tmp_stk_returns = stock_returns.loc[tmp_complete_position.index[0]:,tmp_complete_position.columns] # 只选择涉及到的日期和基金
            tmp_complete_position = tmp_complete_position/100 # 权重小数化
            tmp_complete_position.loc[tmp_stk_returns.index[-1],:] = np.nan
            
            daily_returns = pd.Series((tmp_stk_returns * (tmp_complete_position.asfreq('B').ffill().shift(1))).sum(1),\
            index = pd.to_datetime(tmp_stk_returns.index), name = 'Returns') # 模拟权重的日度收益率
            
            # 读取真实的后复权净值
            real_fund_info = ChinaMutualFundNAV.query("F_INFO_WINDCODE == '{}'".format(tmp_fund))
            real_fund_nav = real_fund_info.pivot(index = "PRICE_DATE",columns = "F_INFO_WINDCODE",values = "F_NAV_ADJUSTED")
            real_fund_nav.index = pd.to_datetime(real_fund_nav.index,format = "%Y%m%d")
            
            # 截尾日期对齐
            if real_fund_info.empty or daily_returns.empty: # 如果净值数据中没有数据，则直接跳过
                continue
            daily_returns = daily_returns.loc[:real_fund_nav.index[-1]]
            
            # 真实的净值序列
            if real_fund_nav.index[0] > daily_returns.index[0]:
                real_fund_nav_series = real_fund_nav.copy()
                daily_returns = daily_returns.loc[real_fund_nav_series.index[0]:] # 净值从可获得的真实净值开始计算
            else: # 中断的净值序列需要向后填充
                real_fund_nav_series = real_fund_nav.loc[daily_returns.index[0]:]/real_fund_nav.loc[daily_returns.index[0]:].ffill().iloc[0]
                
            # 计算spearman相关系数
            spearman_corr = Decimal(spearmanr(pd.concat([np.cumprod((1+daily_returns)),real_fund_nav_series],axis = 1).dropna()).correlation).quantize(Decimal("0.0000")) 
            # 作图
            fig = plt.figure(figsize = (18,6),dpi = 150)
            ax1 = fig.add_subplot(111)
            ax1.plot(np.cumprod((1 + daily_returns)), "-",color = "#0000FF",linewidth = 1)#非预测对比序列
            ax1.plot(real_fund_nav_series, "-",color = "r",linewidth = 1,)#预测对比序列
            plt.xticks(fontproperties='Times New Roman', size = 11.5)
            plt.yticks(fontproperties='Times New Roman', size = 11.5)

            plt.legend(["Stimulated NAV of {}".format(tmp_fund),"Real NAV of {}".format(tmp_fund)], loc='upper left',fontsize = 11.5,shadow = True)
            ax1.set_ylabel("Net Value (Adjusted)",fontsize = 12,weight = 'bold')
            ax1.set_xlabel("Years / Spearman Corr:{}".format(spearman_corr),fontsize = 12,weight = 'bold')
            
            plt.savefig("./Output/NavBiasPlot/{}.png".format(tmp_fund[:6])) # 保存偏误图片
            plt.close()

processing 000006.OF
processing 000017.OF
processing 000020.OF
processing 000021.OF
processing 000031.OF
processing 000039.OF
processing 000057.OF
processing 000061.OF
processing 000082.OF
processing 000083.OF
processing 000117.OF
processing 000120.OF
processing 000124.OF
processing 000127.OF
processing 000172.OF
processing 000173.OF
processing 000199.OF
processing 000209.OF
processing 000220.OF
processing 000251.OF
processing 000263.OF
processing 000294.OF
processing 000308.OF
processing 000309.OF
processing 000326.OF
processing 000327.OF
processing 000339.OF
processing 000354.OF
processing 000409.OF
processing 000411.OF
processing 000418.OF
processing 000432.OF
processing 000457.OF
processing 000471.OF
processing 000513.OF
processing 000522.OF
processing 000524.OF
processing 000532.OF
processing 000547.OF
processing 000549.OF
processing 000550.OF
processing 000551.OF
processing 000577.OF
processing 000586.OF
processing 000589.OF
processing 000591.OF
processing 000592.OF
processing 00

processing 005875.OF
processing 005888.OF
processing 005894.OF
processing 005904.OF
processing 005911.OF
processing 005914.OF
processing 005927.OF
processing 005937.OF
processing 005939.OF
processing 005960.OF
processing 005962.OF
processing 005967.OF
processing 005968.OF
processing 005970.OF
processing 005983.OF
processing 006002.OF
processing 006007.OF
processing 006022.OF
processing 006025.OF
processing 006049.OF
processing 006058.OF
processing 006072.OF
processing 006081.OF
processing 006084.OF
processing 006106.OF
processing 006111.OF
processing 006113.OF
processing 006121.OF
processing 006122.OF
processing 006123.OF
processing 006128.OF
processing 006132.OF
processing 006136.OF
processing 006138.OF
processing 006154.OF
processing 006157.OF
processing 006160.OF
processing 006167.OF
processing 006179.OF
processing 006193.OF
processing 006195.OF
processing 006199.OF
processing 006201.OF
processing 006202.OF
processing 006205.OF
processing 006218.OF
processing 006223.OF
processing 00

processing 009137.OF
processing 009138.OF
processing 009140.OF
processing 009141.OF
processing 009147.OF
processing 009152.OF
processing 009162.OF
processing 009188.OF
processing 009189.OF
processing 009190.OF
processing 009199.OF
processing 009200.OF
processing 009210.OF
processing 009223.OF
processing 009234.OF
processing 009240.OF
processing 009242.OF
processing 009246.OF
processing 009263.OF
processing 009264.OF
processing 009265.OF
processing 009312.OF
processing 009318.OF
processing 009327.OF
processing 009330.OF
processing 009334.OF
processing 009341.OF
processing 009342.OF
processing 009347.OF
processing 009353.OF
processing 009360.OF
processing 009362.OF
processing 009364.OF
processing 009366.OF
processing 009368.OF
processing 009376.OF
processing 009379.OF
processing 009380.OF
processing 009381.OF
processing 009391.OF
processing 009394.OF
processing 009398.OF
processing 009402.OF
processing 009411.OF
processing 009414.OF
processing 009432.OF
processing 009437.OF
processing 00

processing 011021.OF
processing 011042.OF
processing 011046.OF
processing 011058.OF
processing 011069.OF
processing 011077.OF
processing 011078.OF
processing 011097.OF
processing 011099.OF
processing 011104.OF
processing 011121.OF
processing 011128.OF
processing 011134.OF
processing 011136.OF
processing 011138.OF
processing 011142.OF
processing 011144.OF
processing 011146.OF
processing 011149.OF
processing 011152.OF
processing 011153.OF
processing 011155.OF
processing 011157.OF
processing 011160.OF
processing 011162.OF
processing 011164.OF
processing 011169.OF
processing 011170.OF
processing 011173.OF
processing 011174.OF
processing 011177.OF
processing 011179.OF
processing 011181.OF
processing 011184.OF
processing 011186.OF
processing 011188.OF
processing 011189.OF
processing 011194.OF
processing 011201.OF
processing 011203.OF
processing 011206.OF
processing 011212.OF
processing 011216.OF
processing 011220.OF
processing 011229.OF
processing 011236.OF
processing 011238.OF
processing 01

processing 519712.OF
processing 519714.OF
processing 519727.OF
processing 519736.OF
processing 519915.OF
processing 519935.OF
processing 519965.OF
processing 519975.OF
processing 519979.OF
processing 519983.OF
processing 519987.OF
processing 519991.OF
processing 519993.OF
processing 519994.OF
processing 519996.OF
processing 530001.OF
processing 530003.OF
processing 530006.OF
processing 530011.OF
processing 530019.OF
processing 540002.OF
processing 540006.OF
processing 540007.OF
processing 540008.OF
processing 540009.OF
processing 540010.OF
processing 550002.OF
processing 550003.OF
processing 550008.OF
processing 550009.OF
processing 560002.OF
processing 570001.OF
processing 570005.OF
processing 570006.OF
processing 570007.OF
processing 570008.OF
processing 580002.OF
processing 580003.OF
processing 580006.OF
processing 580008.OF
processing 590001.OF
processing 590002.OF
processing 590008.OF
processing 610001.OF
processing 610004.OF
processing 610005.OF
processing 610006.OF
processing 61